In [10]:
import numpy as np
from numba import njit, prange
from compmemlearn.models import Classic_CMR

@njit(fastmath=True, nogil=True, parallel=True)
def cmr_rep_likelihood(
        trials, presentations, list_length, encoding_drift_rate, start_drift_rate, 
        recall_drift_rate, shared_support, item_support, learning_rate, 
        primacy_scale, primacy_decay, stop_probability_scale, 
        stop_probability_growth, choice_sensitivity, delay_drift_rate, drift_familiarity_scale, 
        mfc_familiarity_scale, mcf_familiarity_scale, sampling_rule):
    """
    Generalized cost function for fitting the InstanceCMR model optimized 
    using the numba library.
    
    Output scales inversely with the likelihood that the model and specified 
    parameters would generate the specified trials. For model fitting, is 
    usually wrapped in another function that fixes and frees parameters for 
    optimization.

    **Arguments**:
    - data_to_fit: typed list of int64-arrays where rows identify a unique 
        trial of responses and columns corresponds to a unique recall index.  
    - A configuration for each parameter of `InstanceCMR` as delineated in 
        `Formal Specification`.

    **Returns** the negative sum of log-likelihoods across specified trials 
    conditional on the specified parameters and the mechanisms of InstanceCMR.
    """

    likelihood = np.ones((len(trials), list_length))

    for trial_index in prange(len(trials)):

        item_count = np.max(presentations[trial_index])+1
        
        model = Classic_CMR(
            item_count, list_length, encoding_drift_rate, start_drift_rate, 
            recall_drift_rate, shared_support, item_support, learning_rate, 
            primacy_scale, primacy_decay, stop_probability_scale, 
            stop_probability_growth, choice_sensitivity, delay_drift_rate,  drift_familiarity_scale,
            mfc_familiarity_scale, mcf_familiarity_scale, sampling_rule)

        model.experience(np.eye(item_count, item_count)[presentations[trial_index]])
            
        trial = trials[trial_index]

        model.force_recall()
        for recall_index in range(len(trial) + 1):

            # identify index of item recalled; if zero then recall is over
            if recall_index == len(trial) and len(trial) < item_count:
                recall = 0
            elif trial[recall_index] == 0:
                recall = 0
            else:
                recall = presentations[trial_index][trial[recall_index]-1] + 1

            # store probability of and simulate recalling item with this index
            likelihood[trial_index, recall_index] = \
                model.outcome_probabilities(model.context)[recall]

            if recall == 0:
                break
            model.force_recall(recall)

        # reset model to its pre-retrieval (but post-encoding) state
        model.force_recall(0)

    return -np.sum(np.log(likelihood))

def cmr_rep_objective_function(data_to_fit, presentations, list_types, list_length, fixed_parameters, free_parameters):
    """
    Generates and returns an objective function for input to support search 
    through parameter space for ICMR model fit using an optimization function.

    Arguments:  
    - fixed_parameters: dictionary mapping parameter names to values they'll 
        be fixed to during search, overloaded by free_parameters if overlap  
    - free_parameters: list of strings naming parameters for fit during search  
    - data_to_fit: array where rows identify a unique trial of responses and 
        columns corresponds to a unique recall index

    Returns a function that accepts a vector x specifying arbitrary values for 
    free parameters and returns evaluation of icmr_likelihood using the model 
    class, all parameters, and provided data.
    """
    return lambda x: cmr_rep_likelihood(data_to_fit, presentations, list_types, list_length, **{**fixed_parameters, **{
        free_parameters[i]:x[i] for i in range(len(x))}})

In [11]:
from compmemlearn.datasets import prepare_lohnas2014_data

import pandas as pd
from psifr import fr
from scipy.optimize import differential_evolution
from numba.typed import List
import numpy as np

trials, events, list_length, presentations, list_types, rep_data, subjects = prepare_lohnas2014_data(
    '../data/repFR.mat')

free_parameters = [
    'encoding_drift_rate',
    'start_drift_rate',
    'recall_drift_rate',
    'shared_support',
    'item_support',
    'learning_rate',
    'primacy_scale',
    'primacy_decay',
    'stop_probability_scale',
    'stop_probability_growth',
    'choice_sensitivity',
    'delay_drift_rate',
    'mfc_familiarity_scale',
    'mcf_familiarity_scale',
    'drift_familiarity_scale']

fit_values = [8.36412380e-01, 1.37438014e-01, 9.14231622e-01, 1.03548867e-01,
 1.00000000e+00, 2.70091435e-01, 3.89549514e+00, 6.04259249e-01,
 2.04970025e-02, 1.20035161e-01, 1.78302334e+00, 9.96231007e-01,
 0.0, 0.0, 0.0]

parameters = {free_parameters[i]:fit_values[i] for i in range(len(fit_values))}
parameters['sampling_rule'] = 0

In [12]:
selection = list_types >= 1

cmr_rep_likelihood(
         trials[selection], presentations[selection], list_length, **parameters)

60356.06992679857

In [14]:
%%timeit
cmr_rep_likelihood(
         trials[selection], presentations[selection], list_length, **parameters)

165 ms ± 8.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


```
Base result: 305 ms ± 7.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Value: 60356.06992679857
```